I was unable to find any large dataset which contained user emails of the type I wanted.
Fitting any clustering algorithm on this dataset was producing satisfactory but not good results
hence, I discarded the idea.
however this notebook still contains much of the work which I tried.

In [13]:
import re

import joblib
import pandas as pd
import scipy.sparse
from sklearn.impute import SimpleImputer

mails_df = pd.read_csv("../data/training_data.csv", sep='~', index_col=0)
imputer = SimpleImputer(strategy='constant', fill_value='')
mails_df = pd.DataFrame(imputer.fit_transform(mails_df), columns=mails_df.columns)
# mails_df = mails_df.iloc[:10]

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [15]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return processed_text


def preprocess_sender(address):
    address_lst = address.lower().split('@')
    address_lst[1] = re.sub('[.]ac|[.]in|[.]com', '', address_lst[1])
    address_lst[1] = re.sub('[.]', ' ', address_lst[1])
    address_lst[0] = re.sub('[._]', '', address_lst[0])
    return ' '.join(address_lst)

preprocessor = ColumnTransformer(transformers=[
    ('subject', TfidfVectorizer(preprocessor=preprocess_text, min_df=0.01), 'subject'),
    ('body', TfidfVectorizer(preprocessor=preprocess_text, max_df=0.8, min_df=0.01), 'body'),
    ('sender', TfidfVectorizer(preprocessor=preprocess_sender), 'sender')
], remainder='drop')

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [16]:
preprocess_sender('hello_me@cs.iitr.ac.in')

'hellome cs iitr'

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.json', 'r') as file:
    all_labels = json.load(file)
label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in mails_df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['CATEGORY_FORUMS', 'CATEGORY_PERSONAL', 'CATEGORY_PROMOTIONS', 'CATEGORY_UPDATES', 'IMPORTANT', 'INBOX', 'SENT', 'STARRED', 'UNREAD'] will be ignored
  warnings.warn(


In [18]:
feature_matrix = pipeline.fit_transform(mails_df)
feature_matrix

<1198x1861 sparse matrix of type '<class 'numpy.float64'>'
	with 75856 stored elements in Compressed Sparse Row format>

In [19]:
X = feature_matrix
y = labels
print(X.shape, y.shape)

(1198, 1861) (1198, 21)


In [20]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

k = 25
model = KMeans(n_clusters=k, random_state=42)
labels = model.fit_predict(X)
labels

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([21,  9,  9, ...,  5, 12, 21])

In [21]:
type(labels)

numpy.ndarray

In [22]:
mails_df['label'] = pd.Series(labels, name='label')

In [23]:
mails_df.loc[:,['subject', 'label']]

,subject,label
0,Cognizance 2023 is Live Now - Join the Excitem...,21
1,Opportunity to the Researchers to write techni...,9
2,Advisory to reach your Bhawan by 1 am on March...,9
3,"Annual Art Exhibition - ""दर्पण"" | Fine Arts | ...",3
4,CCF stall with fun games and merchandise in AB...,8
...,...,...
1193,Guest lecture by Dr. Anil Prakash Joshiji || N...,21
1194,Dead & Alive Presentations,8
1195,Re: Regarding the updated mobile no. and room no.,5
1196,Updates: Your List of Quarantined Emails since...,12
